#### Classify icebergs and ships on the SAR image

In [1]:
import os
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from os.path import join
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

# import sys
# sys.path.append(r'C:/Users/arman/Documents/GitHub/sar_ship_detect/scripts')

In [2]:
# Define path to the data
# PATH = r'C:\Users\arman\Documents\GitHub\sar_ship_detect'
PATH = '/home/manvel/GitHub/sar_ship_detect/'
#data_path = join(os. getcwd(), '..','data', 'Iceberg-classifier-challenge')
DATA_PATH = join(PATH, 'data/Iceberg-classifier-challenge')
OUT_PATH = join(PATH, 'model_weights')

In [3]:
#Load the data.
train = pd.read_json(join(DATA_PATH, 'train/train.json'))
#test = pd.read_json(join(DATA_PATH, 'test/test.json')) # test doesn't include the ship class data

In [4]:
print(train.columns)
print(train.shape)
print(train.groupby('is_iceberg').size())

Index(['id', 'band_1', 'band_2', 'inc_angle', 'is_iceberg'], dtype='object')
(1604, 5)
is_iceberg
0    851
1    753
dtype: int64


### Intro about the Data.

**This primary code was found in this [Kaggle](https://www.kaggle.com/manvelkhudinyan/keras-model-for-beginners-0-210-on-lb-eda-r-d/edit) notebook**

**NOTE**: `not all the modifications here are mentioned, so you should find youeself if needed`

Sentinet -1 sat is at about 680 Km above earth. Sending pulses of signals at a particular angle of incidence and then recoding it back. Basically those reflected signals are called backscatter. The data we have been given is backscatter coefficient which is the conventional form of backscatter coefficient given by:

$ σo (dB) = βo (dB) + 10log10 [ sin(ip) / sin (ic)] $

where
1. ip=is angle of incidence for a particular pixel
2. 'ic' is angle of incidence for center of the image
3. K =constant.

We have been given $σo$ directly in the data. 
#### Now coming to the features of **σo** (sigma naught)\
Basically σo varies with the surface on which the signal is scattered from. For example, for a particular angle of incidence, it varies like:
*             WATER...........           SETTLEMENTS........           AGRICULTURE...........          BARREN........

1.**HH:**     -27.001   ................                     2.70252       .................                -12.7952        ................    -17.25790909

2.**HV:**      -28.035      ................            -20.2665             ..................          -21.4471       .................     -20.019

As you can see, the HH component varies a lot but HV doesn't.
**I don't have the data for scatter from ship, but being a metal object, it should vary differently as compared to ice object.**

Now coming to features, for the purpose of this demo code, all two bands were included and taking avg of them as 3rd channel to create a 3-channel RGB equivalent. 

### original version

<!-- Generate the training data
Create 3 bands having HH, HV and avg of both -->
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

data = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],
                          ((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
target=train['is_iceberg']

## modified

#### The band of angle has missing values
needs to be filled

The dataset explorative analysis show that a single incidence angle is presented for each ship chip. Thus, we can recover the image whole images with that given value in order to process in the model as another band for training.

As the incidence angle range for IW scanning mode (Interferometric Wide Swath) alters in the range of *29°-46°*, we can replace the missing incidence angles with **36.0°** (approximate average). More information on the Sentinel 1 IW mode can be found [here](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/acquisition-modes/interferometric-wide-swath)

Also, some articles state the importance of ratio of **Dual-Polarization incoheent** channels for object detection on sea. Hence we calculate also **HH/HV ratio** as anothee band

In [5]:
#imputer = SimpleImputer()
# simply replace all the 'na' values with a fixed value
train = train.replace({"inc_angle": 'na'}, 36.0)
# recover the image size for incidence angle band
train["inc_angle"] = train["inc_angle"].apply(lambda x: [x]*len(train["band_1"][0]))

In [6]:
# modify the data preprocessing
#Create 3 bands having HH, HV and incidence angle
HH = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
HV = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
angle = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["inc_angle"]])
Ratio = HH/HV # just to bring it back to the scale of the HH and HV data in dB

data = np.concatenate([HH[:, :, :, np.newaxis], HV[:, :, :, np.newaxis],
                          angle[:, :, :, np.newaxis]], axis=-1)
# data = Ratio[:, :, :, np.newaxis]

target = train['is_iceberg']

## original
`a fancy way of visualization which takes lots of RAM memory -> better not to run for the main analysis`

In [7]:
#Take a look at a iceberg
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
def plotmy3d(c, name):

    data = [
        go.Surface(
            z=c
        )
    ]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=700,
        height=700,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
plotmy3d(HH[12,:,:], 'iceberg')

In [8]:
plotmy3d(HH[14,:,:], 'Ship')

In [9]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.metrics import binary_accuracy

Using TensorFlow backend.


In [10]:
#original model
#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 2
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

#file_path = ".model_weights.hdf5"
#file_path = ".model_weights_inc_angle_as_band.hdf5"
MODEL_NAME = "..model_weights_Ice_inc_angle_as_band.hdf5"
weigth_path = join(OUT_PATH, MODEL_NAME)
callbacks = get_callbacks(filepath=join(OUT_PATH, MODEL_NAME ), patience=10)

### Train-test split

original implementation

target=train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=1, train_size=0.75)

## modified split

In [11]:
# Define the data split strategy
def data_split(data, target, train_size, test_size): 
    ''' Train-validation-test split'''

    # split data to get the initial training test split
    X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=1, 
                                                train_size=train_size, stratify = target)
    
    # split data to get train validation split
    X_train_cv, X_valid, y_train_cv, y_valid = train_test_split( X_train, y_train, random_state=1,
                                                test_size = valid_size, stratify = y_train) 
    
    return  X_train_cv, X_valid, X_test, y_train_cv, y_valid, y_test

# define split parameters
train_size = 0.90
valid_size = 0.20

X_train_cv, X_valid, X_test, y_train_cv, y_valid, y_test = data_split(data, target, train_size, valid_size )
print(f'data split: \nTrain: \t   {X_train_cv.shape[0]} \nValidation: {X_valid.shape[0]} \nTest: \t    {X_test.shape[0]}')

data split: 
Train: 	   1154 
Validation: 289 
Test: 	    161


In [12]:
#Without denoising, core features.
gmodel=getModel()
gmodel.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=50,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)      

1154/1154 [==============================] - 14s 12ms/step - loss: 0.2430 - accuracy: 0.8761 - val_loss: 0.2410 - val_accuracy: 0.8858
Epoch 36/50
1154/1154 [==============================] - 14s 12ms/step - loss: 0.2587 - accuracy: 0.8830 - val_loss: 0.2486 - val_accuracy: 0.8962
Epoch 37/50
1154/1154 [==============================] - 14s 12ms/step - loss: 0.2853 - accuracy: 0.8718 - val_loss: 0.2391 - val_accuracy: 0.8893
Epoch 38/50
1154/1154 [==============================] - 14s 12ms/step - loss: 0.2590 - accuracy: 0.9029 - val_loss: 0.2547 - val_accuracy: 0.8962
Epoch 39/50
1154/1154 [==============================] - 14s 12ms/step - loss: 0.2416 - accuracy: 0.8856 - val_loss: 0.2970 - val_accuracy: 0.8581
Epoch 40/50
1154/1154 [==============================] - 14s 12ms/step - loss: 0.2433 - accuracy: 0.8917 - val_loss: 0.2431 - val_accuracy: 0.8893
Epoch 41/50
1154/1154 [==============================] - 14s 12ms/step - loss: 0.2372 - accuracy: 0.9012 - val_loss: 0.2828 - val_

In [13]:
# model validation evaluation
gmodel.load_weights(weigth_path)
score = gmodel.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

289/289 [==============================] - 1s 3ms/step
Test loss: 0.23914480828083923
Test accuracy: 0.8892733454704285


#### Prediction

In [14]:
# model test evaluation
score = gmodel.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

161/161 [==============================] - 1s 3ms/step
Test loss: 0.2608347891924631
Test accuracy: 0.8757764101028442


## modified
#### Using sklearn library to calculate prediction accuracy

from sklearn.metrics import accuracy_score

y_test_transf = np.reshape(y_test.values, y_test.shape[0])

y_pred = np.reshape(predicted_test, y_test.shape[0])
y_pred = np.array([int(round(x)) for x in y_pred])

acc_sk = accuracy_score(y_test_transf, y_pred)
print(f'Iceberg detection accuracy: {round(acc_sk*100, 2)} %')